#### PipelineRunner

In [ ]:
object PipelineRunner {

  def run(
      pipelineName: String,
      queryConfigPath: String,
      transformFunc: DataFrame => DataFrame,
      targetTable: String,
      dryRun: Boolean = false
  ): Unit = {

    // 1. Start the checkpoint
    Checkpoints.markRunStarted(pipelineName)

    try {
      // 2. Extract
      val rawDF = DeltaReader.readDeltaWithWatermark(queryConfigPath)
      if (rawDF.isEmpty) {
        println(s"No data found for $pipelineName")
        Checkpoints.markCheckpointSkipped(pipelineName)
        return
      }

      // 3. Transform
      val processedDF = transformFunc(rawDF)

      // 4. Load
      if (!dryRun) {
        DeltaWriter.writeDelta(processedDF, targetTable)
      }

      // 5. Mark success
      Checkpoints.markRunCompleted(pipelineName)

    } catch {
      case e: Throwable =>
        // 6. Mark failure
        Checkpoints.markRunFailed(pipelineName, Some(e.getMessage))
        throw e
    }
  }
}


#### Example

In [ ]:

// A standard pipelien workflow for a single downstream DAP table
// the workdflow can be implemented in pipeline core proces workflow 

object AuthorprofilePipeline {

  /* Setion-1: define functions to for business logic */

    // def function-1 ...
    // def function-2 ...
    // def function-3 ...

  // task-1: porcess entity data  and related attributes
  def processEntity() {
    // logic to process meta data for entity
   }

  // task-2: porcess article level metric data
  def processEntityMetrics() {
      // logic to process metric data for entity
  }
}


object Main extends App {

  PipelineRunner.run(
    pipelineName     = "EntityPipeline",
    queryConfigPath  = "config/ap_incremental_sql.yml",
    transformFunc    = AuthorprofilePipeline.processEntity,
    targetTable      = "dap_entity_wos.authorprofile"
  )

  PipelineRunner.run(
    pipelineName     = "EntityMetricsPipeline",
    queryConfigPath  = "config/ap_metrics_sql.yml",
    transformFunc    = Transformations.processEntityMetrics,
    targetTable      = "dap_entity_metrics.authorprofile"
  )
}

